In [ ]:
# Takes in a given comparison file (in the correct format), and calculates the KCRV and DoEs using Bayesian model averaging. 
# The model order parameter used is m=lchs-1. This can be changed by editing the value of m in the fifth cell. Note that BMA is only designed to work in the scenario where there is only one artefact.

# Returns an excel file with the measurements, uncertainties, DoEs and KCRVs.

code_version = 1

In [1]:
import xlrd
import math
import xlwt
from scipy import stats
import numpy as np
from scipy.stats import chi2
import operator as op
import itertools

In [27]:
# Run the methods and read data files so we can call those functions
%run "Methods-Using-GLS.ipynb" -G

In [15]:
# Specify the directory in which the comparison data files are saved
# List all files which you want to analyse
files = ["CCL-K1-1.xls"]

In [31]:
for file in files:

    # Read data in from file
    [y, U, NumArtefacts, NumMeasurements, NumParticipants, ave_U, X, w,ParticipantNames,MeasurementTitles,
     count_per_lab, ArtefactNames] = read_data(file)

    # Create workbook to save data
    wb = xlwt.Workbook()
    
    # Add sheet with measurements
    ws = wb.add_sheet('y')
    ws.write(0,0,'Participant')
    ws.write(0,1,'Measurement')
    ws.write(0,2,'Uncertainty (k=1)')
    for i in range(0,NumMeasurements):
        ws.write(i+1,1,y[i,0])
        ws.write(i+1,0,ParticipantNames[i])
        ws.write(i+1,2,ave_U[i])
        
    
    ws.write(5,0,'Code version ' + str(code_version))
    
    # Calculate beta, CovBeta
    [gamma, beta, CovBeta] = calculate_beta(y, U, X, w)
        
    lchs = LChS(beta, CovBeta)
    
    m = lchs - 4
    
    [alpha, unc_alpha, mu, unc_mu] = BMA(y,U,m)

    # Add sheet with differences
    ws = wb.add_sheet('DoEs')
    ws.write(0,0,'Participant')
    ws.write(0,1,'DoE')
    ws.write(0,2,'U(DoE)')
    for i in range(0,NumParticipants):
        ws.write(i+1,0,ParticipantNames[i])
        ws.write(i+1,1,alpha[i])
        ws.write(i+1,2,unc_alpha[i])

    # Add sheet with the theta
    ws = wb.add_sheet('Theta')
    ws.write(0,0,'Artefact')
    ws.write(0,1,'KCRV')
    ws.write(0,2,'U(KCRV)')
    ws.write(1,0,ArtefactNames[0])
    ws.write(1,1,mu[0])
    ws.write(1,2,unc_mu[0])
    

    wb.save(file+'-BMA.xls')